# 0. Imports, libraries and rusable functions

In [5]:
from project_imports import *
import use_gpu
# Clear any cached memory to start fresh for each trial
torch.cuda.empty_cache()
gc.collect()

63

# 1. Global Variables

In [6]:
## Arguments and global vriables
dataset_name="MCQA-Combined-9"
global_run_name="Optuna-1"
#pretrained_model_name = "microsoft/deberta-v3-base"
pretrained_model_name = "sentence-transformers/all-mpnet-base-v2"
normalized_model_name = pretrained_model_name.replace("/", "-")
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
assert isinstance( tokenizer, PreTrainedTokenizerFast )
data_collator = DefaultDataCollator()
max_length = 512 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
pad_on_right = right_padding = tokenizer.padding_side == 'right'
global_counter = 0
traing_answer_mismatches = []
logger = logging.getLogger(__name__)

# 2. Prepare the Dataset 

In [7]:
# Load the combined dataset
combined_dataset = load_from_disk('cleaned_dataset')

combined_dataset

DatasetDict({
    train: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 1072514
    })
    validation: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 118521
    })
    test: Dataset({
        features: ['Context', 'Question', 'Options', 'Label_Text', 'Label', 'Type', 'Source Dataset'],
        num_rows: 200566
    })
})

In [8]:
# Initialize lists to hold datasets
train_datasets = []
val_datasets = []
test_datasets = []

# List of datasets to combine
datasets_to_combine = ['AR-LSAT', 'ReClor', 'LogiQA 2.0', 'RTE', 'FOLIO', 'PrOntoQA', 'MRPC', 'Adversarial NLI', 'ConTRoL' ]

# Loop through each dataset and filter
for ds_name in datasets_to_combine:
    train_datasets.append(combined_dataset['train'].filter(lambda x: x['Source Dataset'] == ds_name))
    val_datasets.append(combined_dataset['validation'].filter(lambda x: x['Source Dataset'] == ds_name))
    test_datasets.append(combined_dataset['test'].filter(lambda x: x['Source Dataset'] == ds_name))

# Concatenate datasets
combined_train = concatenate_datasets(train_datasets)
combined_val = concatenate_datasets(val_datasets)
combined_test = concatenate_datasets(test_datasets)

# Concatenate validation data into the training data and use the Test dataset for validation 
combined_train = concatenate_datasets([combined_train, combined_val])
combined_val = combined_test

# Shuffle the combined dataset
# To ensure that each training batch has a chance to contain a mix of examples from all sources. 
# This helps in reducing variance and improving the generalization of the model.
combined_train = combined_train.shuffle(seed=42)

In [9]:
def mcqa_preprocess_function(examples):
    # Determine the maximum number of choices
    max_num_choices = 5  # Since AR-LSAT has 5 options, we'll pad others to 5
    contexts = examples['Context']
    questions = examples['Question']
    options_list = examples['Options']
    labels = examples['Label']
    
    first_sentences = []
    second_sentences = []
    labels_adjusted = []
    
    for context, question, options, label in zip(contexts, questions, options_list, labels):
        num_choices = len(options)
        # Pad options to have max_num_choices
        if num_choices < max_num_choices:
            options += [''] * (max_num_choices - num_choices)
        first_sentences.append([context] * max_num_choices)
        second_sentences.append([f"{question} {option}" for option in options])
        labels_adjusted.append(label)
    
    # Flatten the lists
    first_sentences = [item for sublist in first_sentences for item in sublist]
    second_sentences = [item for sublist in second_sentences for item in sublist]
    
    # Tokenize the inputs
    tokenized_examples = tokenizer(
        first_sentences,
        second_sentences,
        truncation=True,
        max_length=512,
        padding='max_length',
    )
    
    # Un-flatten to shape (num_examples, max_num_choices, seq_length)
    tokenized_inputs = {
        k: [v[i:i + max_num_choices] for i in range(0, len(v), max_num_choices)]
        for k, v in tokenized_examples.items()
    }
    
    # Labels
    tokenized_inputs["labels"] = labels_adjusted
    
    return tokenized_inputs




In [10]:
# Apply the preprocessing function to the combined datasets
encoded_train = combined_train.map(mcqa_preprocess_function, batched=True)
encoded_val = combined_val.map(mcqa_preprocess_function, batched=True)
encoded_test = combined_test.map(mcqa_preprocess_function, batched=True)


Map:   0%|          | 0/36630 [00:00<?, ? examples/s]

Map:   0%|          | 0/5569 [00:00<?, ? examples/s]

In [11]:
# Set the format of the datasets to PyTorch tensors
encoded_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
encoded_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])


def get_train_encoded():
    return encoded_train

def get_val_encoded():
    return encoded_val

def get_test_encoded():
    return encoded_test


print("Number of training examples:", len(encoded_train))
print("Number of validation examples:", len(encoded_val))
print("Number of test examples:", len(encoded_test))

Number of training examples: 36630
Number of validation examples: 5569
Number of test examples: 5569


# 3. Reusable Functions

In [12]:
# Load the accuracy metric
accuracy = evaluate.load('accuracy')

# Define the compute_metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1 = f1_score(labels, predictions, average='weighted')
    return {'eval_accuracy': acc, 'eval_f1': f1}

In [13]:
def create_training_args(run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=3):
    """
    Generates training arguments for training a machine learning model.

    Parameters:
    - dataset_name (str): The name of the dataset.
    - run_name (str): The name of the run, useful for logging and saving models.
    - model_name (str): The name of the model, typically including its configuration.
    - num_train_epochs (int): The number of epochs to train for.
    - learning_rate (float): The learning rate for training.
    - batch_size (int): The batch size used for training.

    Returns:
    - TrainingArguments: A configured TrainingArguments instance.
    """    
    output_dir = f"./{dataset_name}/{run_name}/{normalized_model_name}"
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        report_to="none",  # Disable all integrations
        overwrite_output_dir=True,
        metric_for_best_model='eval_accuracy',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=num_train_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=4,
        warmup_steps=398,
        weight_decay=0.194,
        adam_beta1=0.837,
        adam_beta2=0.997,
        adam_epsilon=5.87e-07,
        lr_scheduler_type='cosine',
        fp16=True,  # Enable mixed-precision training
    )
    
    return training_args


In [14]:
def model_init(model_name=pretrained_model_name, dropout_rate=0.1):
    model = AutoModelForMultipleChoice.from_pretrained(model_name)
    #model.config.hidden_dropout_prob = dropout_rate
    return model

In [15]:
def create_trainer(model_name=pretrained_model_name,run_name="Default-Run", num_train_epochs=3, learning_rate=4.92e-05, batch_size=4):
    trainer = Trainer(
        model=model_init(model_name),
        args=create_training_args(run_name=run_name, num_train_epochs=num_train_epochs, learning_rate=learning_rate, batch_size=batch_size),
        train_dataset=get_train_encoded(),
        eval_dataset=get_val_encoded(),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )
    
    return trainer


In [16]:
class AdvancedEarlyStoppingCallback(TrainerCallback):
    """
    A callback to stop training when either the performance falls below a certain threshold
    or if there is no improvement over a set number of epochs.
    """
    def __init__(self, metric_name, patience):
        self.metric_name = metric_name
        self.patience = patience        
        self.best_score = None
        self.no_improve_epochs = 0
        self.config_file = "early_stopping_config.json"  # Config file for early stopping values

    def read_early_stopping_config(self):
        """
        Reads the early stopping configuration from the file system.
        Returns the configuration as a dictionary.
        """
        if os.path.exists(self.config_file):
            with open(self.config_file, 'r') as file:
                config = json.load(file)
            return config
        else:
            raise FileNotFoundError(f"Config file not found: {self.config_file}")
    def reset_manual_stop_flag(self):
        """
        Resets the manual stop flag to False in the early stopping config file.
        """
        config = self.read_early_stopping_config()
        config['manual_stop'] = False
        with open(self.config_file, 'w') as file:
            json.dump(config, file, indent=4)

    def on_evaluate(self, args, state, control, **kwargs):
        metric_value = kwargs['metrics'].get(self.metric_name)

        if self.best_score is None or metric_value > self.best_score:
            self.best_score = metric_value
            self.no_improve_epochs = 0
        else:
            self.no_improve_epochs += 1

        # Check if no improvement has been seen over the allowed patience
        if self.no_improve_epochs >= self.patience:
            control.should_training_stop = True
            print(f"Stopping training: No improvement in {self.metric_name} for {self.patience} epochs")


        # Read the early stopping configuration
        config = self.read_early_stopping_config()
        min_accuracy = config.get("min_accuracy", 0.35)                
        num_epochs_min_acc = config.get("num_epochs_min_acc", 2)  
        max_variance = config.get("max_variance", 0.2)  

        # Check if performance is below the threshold
        if metric_value < min_accuracy:
            control.should_training_stop = True
            print(f"Stopping training: {self.metric_name} below manual min_acc of {min_accuracy}")

         # Manual stop from config
        if config.get("manual_stop", False):
            control.should_training_stop = True
            print(f"Manual early stopping triggered!!")
            self.reset_manual_stop_flag()  # Reset the flag for future runs
            


In [24]:
# Optuna objective function for hyperparameter tuning
def objective(trial):
    # Clear any cached memory to start fresh for each trial
    torch.cuda.empty_cache()
    gc.collect()

    
    model_name = trial.suggest_categorical('model_name', [pretrained_model_name])     
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-7, 1e-4, log=True)
    batch_size = trial.suggest_categorical('batch_size', [3, 4])
    #warmup_steps = trial.suggest_int('warmup_steps', 0, 1000)
    warmup_ratio= trial.suggest_float('warmup_ratio', 0.0, 1.0)
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.25)
    adam_beta1 = trial.suggest_float('adam_beta1', 0.8, 0.95)
    adam_beta2 = trial.suggest_float('adam_beta2', 0.990, 0.999)
    adam_epsilon = trial.suggest_float('adam_epsilon', 1e-8, 1e-6)
    lr_scheduler_type = trial.suggest_categorical('lr_scheduler_type', ['linear', 'cosine', 'cosine_with_restarts']) #,'constant_with_warmup'   
    

    output_dir = f"./{dataset_name}/{global_run_name}/trial_{trial.number}"

    training_args = TrainingArguments(
        output_dir=output_dir,
        report_to="none",  # Disable all integrations
        overwrite_output_dir=True,
        metric_for_best_model='eval_accuracy',
        greater_is_better=True,
        load_best_model_at_end=True,
        save_total_limit=3,
        eval_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=30,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_ratio=warmup_ratio,
        weight_decay=weight_decay,
        adam_beta1=adam_beta1,
        adam_beta2=adam_beta2,
        adam_epsilon=adam_epsilon,
        lr_scheduler_type=lr_scheduler_type,
        fp16=True,  # Enable mixed-precision training
    ) 
    
    # Print trial parameters
    print(f"Current Trial {trial.number} parameters: {trial.params}")
    
    trainer = Trainer(
        model=model_init(model_name, dropout_rate),
        args=training_args,
        train_dataset=get_train_encoded(),
        eval_dataset=get_val_encoded(),
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[AdvancedEarlyStoppingCallback(metric_name='eval_accuracy', patience=1)]
    )

    # Train the model
    trainer.train()

    # Evaluate the model
    eval_results = trainer.evaluate()
        
    torch.cuda.empty_cache()  # Clear cache after evaluation
    gc.collect()  # Collect garbage

    return eval_results['eval_accuracy']


# 4. DeBERTa Joint MTL Training on the Dataset

## 4.1 Optuna Hyperparameters Tuning 1

In [14]:
# Create a study object and optimize the objective
global_run_name="Optuna-1"
study = optuna.create_study(direction='maximize')
study.enqueue_trial({'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.0052258285035737e-05, 'batch_size': 3, 'warmup_ratio': 0.04149176551014211, 'weight_decay': 0.006685281279171756, 'adam_beta1': 0.9429922176765829, 'adam_beta2': 0.9918592948813898, 'adam_epsilon': 8.867767549079712e-08, 'lr_scheduler_type': 'cosine_with_restarts'})
#study.enqueue_trial({'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'})
study.optimize(objective, n_trials=20)


[I 2024-10-20 19:13:54,181] A new study created in memory with name: no-name-7683d3f0-e950-4b81-96ad-469024148d0e


Current Trial 0 parameters: {'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.0052258285035737e-05, 'batch_size': 3, 'warmup_ratio': 0.04149176551014211, 'weight_decay': 0.006685281279171756, 'adam_beta1': 0.9429922176765829, 'adam_beta2': 0.9918592948813898, 'adam_epsilon': 8.867767549079712e-08, 'lr_scheduler_type': 'cosine_with_restarts'}


Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/366300 [00:00<?, ?it/s]

{'loss': 1.6092, 'grad_norm': 2.5615124702453613, 'learning_rate': 3.306881467542515e-07, 'epoch': 0.04}
{'loss': 1.6037, 'grad_norm': 3.1979482173919678, 'learning_rate': 6.607149172149945e-07, 'epoch': 0.08}
{'loss': 1.5095, 'grad_norm': 3.0810492038726807, 'learning_rate': 9.90080311382229e-07, 'epoch': 0.12}
{'loss': 1.3245, 'grad_norm': 8.836233139038086, 'learning_rate': 1.320107081842972e-06, 'epoch': 0.16}
{'loss': 1.2577, 'grad_norm': 21.913860321044922, 'learning_rate': 1.650133852303715e-06, 'epoch': 0.2}
{'loss': 1.2258, 'grad_norm': 18.503393173217773, 'learning_rate': 1.9808219990579663e-06, 'epoch': 0.25}
{'loss': 1.217, 'grad_norm': 52.439456939697266, 'learning_rate': 2.311510145812218e-06, 'epoch': 0.29}
{'loss': 1.1845, 'grad_norm': 15.72999382019043, 'learning_rate': 2.6421982925664694e-06, 'epoch': 0.33}
{'loss': 1.1741, 'grad_norm': 21.107629776000977, 'learning_rate': 2.9728864393207205e-06, 'epoch': 0.37}
{'loss': 1.1737, 'grad_norm': 25.750137329101562, 'learni

...

Stopping training: No improvement in eval_accuracy for 1 epochs
Stopping training: eval_accuracy below manual min_acc of 0.5
Current Trial 7 parameters: {'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.6467178725088624e-07, 'batch_size': 3, 'warmup_ratio': 0.6518719890876207, 'weight_decay': 0.010273108046471219, 'adam_beta1': 0.9129492559268905, 'adam_beta2': 0.9901934455902108, 'adam_epsilon': 7.518451468450438e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/366300 [00:00<?, ?it/s]

[W 2024-10-22 01:35:16,958] Trial 7 failed with parameters: {'model_name': 'microsoft/deberta-v3-base', 'learning_rate': 1.6467178725088624e-07, 'batch_size': 3, 'warmup_ratio': 0.6518719890876207, 'weight_decay': 0.010273108046471219, 'adam_beta1': 0.9129492559268905, 'adam_beta2': 0.9901934455902108, 'adam_epsilon': 7.518451468450438e-07, 'lr_scheduler_type': 'cosine_with_restarts'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\OEM\AppData\Local\Temp\ipykernel_22244\84796339.py", line 59, in objective
    trainer.train()
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 1938, in train
    return inner_training_loop(
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 2279, in _inn

KeyboardInterrupt: 

## 4.3 Optuna Hyperparameters Tuning 2

In [25]:
# Create a study object and optimize the objective for pretrained_model_name = "sentence-transformers/all-mpnet-base-v2"
global_run_name="Optuna-2"
study = optuna.create_study(direction='maximize')
study.enqueue_trial({'model_name': pretrained_model_name, 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'})
study.optimize(objective, n_trials=20)


[I 2024-10-22 01:57:26,347] A new study created in memory with name: no-name-198a630a-9e5c-4ede-ae18-36aabe3fac1a


Current Trial 0 parameters: {'model_name': 'sentence-transformers/all-mpnet-base-v2', 'dropout_rate': 0.15718330934471478, 'learning_rate': 1.5807103066634623e-05, 'batch_size': 4, 'warmup_ratio': 0.5994150649377659, 'weight_decay': 0.12506835879573128, 'adam_beta1': 0.8136227307274486, 'adam_beta2': 0.9924116710027883, 'adam_epsilon': 1.9858068243318367e-07, 'lr_scheduler_type': 'cosine_with_restarts'}


Some weights of MPNetForMultipleChoice were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/274740 [00:00<?, ?it/s]

{'loss': 1.6088, 'grad_norm': 0.9182241559028625, 'learning_rate': 4.780025580617451e-08, 'epoch': 0.05}
{'loss': 1.6088, 'grad_norm': 0.7863313555717468, 'learning_rate': 9.569649606175901e-08, 'epoch': 0.11}
{'loss': 1.6093, 'grad_norm': 0.7911553978919983, 'learning_rate': 1.4368872076675347e-07, 'epoch': 0.16}
{'loss': 1.6085, 'grad_norm': 0.8689716458320618, 'learning_rate': 1.91488976572928e-07, 'epoch': 0.22}
{'loss': 1.6076, 'grad_norm': 0.8607678413391113, 'learning_rate': 2.3948120127792246e-07, 'epoch': 0.27}
{'loss': 1.6065, 'grad_norm': 1.0950431823730469, 'learning_rate': 2.8737744153350694e-07, 'epoch': 0.33}
{'loss': 1.6024, 'grad_norm': 1.1426678895950317, 'learning_rate': 3.3536966623850147e-07, 'epoch': 0.38}
{'loss': 1.5868, 'grad_norm': 1.5567299127578735, 'learning_rate': 3.8336189094349596e-07, 'epoch': 0.44}
{'loss': 1.5336, 'grad_norm': 2.8619110584259033, 'learning_rate': 4.3135411564849044e-07, 'epoch': 0.49}
{'loss': 1.4498, 'grad_norm': 3.4570772647857666, 

...

Some weights of MPNetForMultipleChoice were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/274740 [00:00<?, ?it/s]

{'loss': 1.6096, 'grad_norm': 0.892684817314148, 'learning_rate': 2.5165542741075785e-10, 'epoch': 0.05}


[W 2024-10-22 12:04:46,078] Trial 6 failed with parameters: {'model_name': 'sentence-transformers/all-mpnet-base-v2', 'dropout_rate': 0.23732993865505386, 'learning_rate': 1.1685926112932391e-07, 'batch_size': 4, 'warmup_ratio': 0.8400211050841219, 'weight_decay': 0.05868908832126296, 'adam_beta1': 0.8519566435750197, 'adam_beta2': 0.9913554074386653, 'adam_epsilon': 7.201851832756769e-07, 'lr_scheduler_type': 'cosine_with_restarts'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\OEM\AppData\Local\Temp\ipykernel_29776\2979044354.py", line 60, in objective
    trainer.train()
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-packages\transformers\trainer.py", line 1938, in train
    return inner_training_loop(
  File "c:\Users\OEM\anaconda3\envs\compsci714win\lib\site-p

KeyboardInterrupt: 

## 4.4 Optuna Hyperparameters Tuning 3

# End of NoteBook